In [4]:
import pandas as pd
from tabulate import tabulate

def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"


def get_stat_df(filename, verbose=True):
    # Sample loading of your CSV - this would be replaced with your actual loading
    result_df = pd.read_csv(filename)

    # metrics = ['mcc', 'sensitivity', 'specificity', 'accuracy', 'precision', 'micro_auroc']
    metrics = ['mcc', 'sensitivity', 'precision', 'micro_auroc']
    grouped = result_df.groupby('model_key').agg({
        k: ['mean', 'std'] for k in metrics
    })

    record_counts = result_df.groupby('model_key').size()

    for metric in metrics:
        grouped[f'{metric}_formatted'] = grouped.apply(lambda row: format_mean_std(row, metric), axis=1)


    # Add the record count column to the grouped dataframe
    grouped['record_count'] = record_counts.values

    formatted_df = grouped[[k + '_formatted' for k in metrics] + ['record_count']]
    formatted_df.columns = metrics + ['record_count']
    formatted_df = formatted_df.sort_values(metrics[0], ascending=True)
    '''
    ESM:
    2: esm-t33-pretrained
    4: esm-33-gearnet-resiboost	-> 2번하고 비교헀을 때: gearnet을 추가하는 게 더 낫다. resiboost를 추가하는 것도 좋다.
    9: esm-33-gearnet-pretrained-freezelm-ensemble -> 4번하고 비교했을 때 pretrain된 거니까 좋다.


    '''
    formatted_df = formatted_df.reset_index()
    
    if verbose:
        markdown_table = tabulate(formatted_df[['model_key'] + metrics] , headers='keys', tablefmt='pipe', showindex=False)
        print(markdown_table)

    return formatted_df



In [5]:
'''
get_stat_df_summary:

'''

def get_stat_df_summary(filename, groups, verbose=True):
    '''
    - can group some model_key together, and select the model with maximum mcc
    - groups is a dict: {group_name: [model_key1, model_key2, ...]}
    '''
    stat_df = get_stat_df(filename, verbose=False)
    
    new_df = pd.DataFrame()
    
    for group_name, model_keys in groups.items():
        group_df = stat_df[stat_df['model_key'].isin(model_keys)]
        if group_df.shape[0] == 0:
            continue
        group_df = group_df.sort_values('mcc', ascending=False)
        group_df = group_df.head(1).copy().reset_index(drop=True)
        group_df.insert(0, 'group_name', group_name)
        new_df = pd.concat([new_df, group_df])
        
    new_df = new_df.sort_values('mcc', ascending=True).reset_index(drop=True)
    
    metrics = ['mcc', 'sensitivity', 'precision', 'micro_auroc']
    if verbose:
        markdown_table = tabulate(new_df[['group_name', 'model_key'] + metrics] , headers='keys', tablefmt='pipe', showindex=False)
        print(markdown_table)
    
    return new_df
        
    
groups = {
    'GearNet': ['gearnet'],
    'ESM': ['esm-t33'],
    'ESM       +              Ensemble': ['esm-t33-ensemble'],
    'ESM       +              Resiboost': [f'esm-t33-rboost{ratio:02d}' for ratio in [5, 10, 25, 50]] + [f'esm-t33-rboost{ratio:02d}-b8' for ratio in [5, 10, 25, 50]],
    'ESM       +            + Boost All': [f'esm-t33-aboost{ratio:02d}' for ratio in [5, 10, 25, 50]],
    'ESM       + Pretrained': ['esm-t33-pretrained', 'esm-t33-pretrained-freezelm'],
    'ESM       + Pretrained + Resiboost': [f'esm-t33-pretrained-rboost{ratio:02d}' for ratio in [5, 10, 25, 50]],
    'ESM       + Pretrained + Ensemble': ['esm-t33-pretrained-ensemble'],
    
    'Multiview': ['esm-33-gearnet'],
    'Multiview +              Ensemble': ['esm-33-gearnet-ensemble'],
    'Multiview +              Resiboost': [f'esm-33-gearnet-rboost{ratio:02d}' for ratio in [5, 10, 25, 50]] + [f'esm-33-gearnet-rboost{ratio:02d}-b8' for ratio in [5, 10, 25, 50]],
    
    'Multiview + Pretrained': ['esm-33-gearnet-pretrained', 'esm-33-gearnet-pretrained-freezelm', 'esm-33-gearnet-pretrained-freezeall'],
    'Multiview + Pretrained + Resiboost': [f'esm-33-gearnet-pretrained-freezelm-rboost{ratio:02d}' for ratio in [5, 10, 25, 50]] + 
                                              [f'esm-33-gearnet-pretrained-rboost{ratio:02d}' for ratio in [10, 50]],
    'Multiview + Pretrained + Boost All': [f'esm-33-gearnet-pretrained-freezelm-aboost{ratio:02d}' for ratio in [5, 10, 25, 50]],
    'Multiview +            + Boost All': [f'esm-33-gearnet-aboost{ratio:02d}' for ratio in [5, 10, 25, 50]],
    'Multiview + Pretrained + Ensemble': [f'esm-33-gearnet-pretrained-ensemble', 'esm-33-gearnet-pretrained-freezelm-ensemble'],
}


get_stat_df_summary('record.csv', groups=groups)


| group_name                        | model_key                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:----------------------------------|:----------------------------|:--------------|:--------------|:--------------|:--------------|
| ESM                               | esm-t33                     | 0.662 ± 0.032 | 0.702 ± 0.043 | 0.675 ± 0.079 | 0.940 ± 0.023 |
| ESM       + Pretrained            | esm-t33-pretrained-freezelm | 0.674 ± 0.019 | 0.684 ± 0.028 | 0.709 ± 0.021 | 0.857 ± 0.025 |
| Multiview +              Ensemble | esm-33-gearnet-ensemble     | 0.709 ± 0.011 | 0.709 ± 0.009 | 0.749 ± 0.017 | 0.913 ± 0.006 |
| Multiview                         | esm-33-gearnet              | 0.719 ± 0.015 | 0.725 ± 0.018 | 0.752 ± 0.026 | 0.910 ± 0.018 |
| Multiview + Pretrained            | esm-33-gearnet-pretrained   | 0.721 ± 0.016 | 0.736 ± 0.021 | 0.743 ± 0.019 | 0.893 ± 0.022 |


,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,ESM,esm-t33,0.662 ± 0.032,0.702 ± 0.043,0.675 ± 0.079,0.940 ± 0.023,5
1,ESM + Pretrained,esm-t33-pretrained-freezelm,0.674 ± 0.019,0.684 ± 0.028,0.709 ± 0.021,0.857 ± 0.025,5
2,Multiview + Ensemble,esm-33-gearnet-ensemble,0.709 ± 0.011,0.709 ± 0.009,0.749 ± 0.017,0.913 ± 0.006,5
3,Multiview,esm-33-gearnet,0.719 ± 0.015,0.725 ± 0.018,0.752 ± 0.026,0.910 ± 0.018,5
4,Multiview + Pretrained,esm-33-gearnet-pretrained,0.721 ± 0.016,0.736 ± 0.021,0.743 ± 0.019,0.893 ± 0.022,5


In [6]:
get_stat_df('imatinib.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('imatinib.csv', groups=groups)

| model_key                                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:--------------------------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet-pretrained-freezeall         | 0.332 ± 0.083 | 0.293 ± 0.057 | 0.482 ± 0.202 | 0.769 ± 0.013 |
| esm-33-gearnet                              | 0.353 ± 0.087 | 0.389 ± 0.107 | 0.437 ± 0.186 | 0.743 ± 0.040 |
| esm-33-gearnet-pretrained-freezelm-aboost05 | 0.358 ± 0.068 | 0.254 ± 0.071 | 0.583 ± 0.107 | 0.793 ± 0.022 |
| esm-33-gearnet-pretrained-freezelm-rboost05 | 0.376 ± 0.025 | 0.509 ± 0.014 | 0.346 ± 0.041 | 0.790 ± 0.009 |
| esm-33-gearnet-pretrained-freezelm          | 0.390 ± 0.059 | 0.435 ± 0.061 | 0.430 ± 0.134 | 0.803 ± 0.021 |
| esm-33-gearnet-pretrained-freezelm-ensemble | 0.403 ± 0.058 | 0.343 ± 0.041 | 0.541 ± 0.099 | 0.806 ± 0.016 |
| esm-t33                                     | 0.429 ± 0.028 | 0.480 ± 0.030 | 0.449 ± 0.062 | 0.826 ± 

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,Multiview,esm-33-gearnet,0.353 ± 0.087,0.389 ± 0.107,0.437 ± 0.186,0.743 ± 0.040,5
1,ESM,esm-t33,0.429 ± 0.028,0.480 ± 0.030,0.449 ± 0.062,0.826 ± 0.031,5
2,Multiview + Ensemble,esm-33-gearnet-ensemble,0.452 ± 0.036,0.302 ± 0.047,0.739 ± 0.069,0.790 ± 0.013,5
3,ESM + Resiboost,esm-t33-rboost10,0.464 ± 0.089,0.435 ± 0.013,0.569 ± 0.172,0.830 ± 0.014,5
4,Multiview + Pretrained,esm-33-gearnet-pretrained,0.479 ± 0.020,0.385 ± 0.045,0.656 ± 0.041,0.760 ± 0.026,5
5,Multiview + Pretrained + Ensemble,esm-33-gearnet-pretrained-ensemble,0.482 ± 0.021,0.322 ± 0.028,0.775 ± 0.015,0.789 ± 0.012,5
6,Multiview + Pretrained + Boost All,esm-33-gearnet-pretrained-freezelm-aboost50,0.485 ± 0.020,0.339 ± 0.036,0.751 ± 0.044,0.803 ± 0.006,5
7,Multiview + + Boost All,esm-33-gearnet-aboost50,0.507 ± 0.042,0.346 ± 0.034,0.794 ± 0.050,0.781 ± 0.019,5
8,ESM + Pretrained + Ensemble,esm-t33-pretrained-ensemble,0.514 ± 0.012,0.400 ± 0.038,0.716 ± 0.046,0.813 ± 0.018,5
9,ESM + Pretrained,esm-t33-pretrained,0.518 ± 0.017,0.424 ± 0.020,0.687 ± 0.044,0.813 ± 0.018,5


In [7]:
get_stat_df('dasatinib.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('dasatinib.csv', groups=groups)

| model_key                                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:--------------------------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet-pretrained-freezelm-aboost05 | 0.565 ± 0.141 | 0.386 ± 0.161 | 0.903 ± 0.082 | 0.914 ± 0.022 |
| esm-33-gearnet-pretrained-freezelm-aboost10 | 0.628 ± 0.060 | 0.460 ± 0.090 | 0.912 ± 0.019 | 0.923 ± 0.015 |
| esm-33-gearnet-pretrained-freezeall         | 0.628 ± 0.077 | 0.514 ± 0.126 | 0.833 ± 0.077 | 0.884 ± 0.014 |
| esm-33-gearnet-aboost10                     | 0.635 ± 0.052 | 0.469 ± 0.067 | 0.911 ± 0.029 | 0.922 ± 0.035 |
| esm-33-gearnet                              | 0.635 ± 0.059 | 0.586 ± 0.070 | 0.744 ± 0.089 | 0.933 ± 0.015 |
| esm-33-gearnet-pretrained-freezelm-ensemble | 0.657 ± 0.020 | 0.506 ± 0.036 | 0.901 ± 0.032 | 0.927 ± 0.029 |
| esm-33-gearnet-ensemble                     | 0.661 ± 0.020 | 0.509 ± 0.036 | 0.904 ± 0.014 | 0.949 ± 

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,Multiview,esm-33-gearnet,0.635 ± 0.059,0.586 ± 0.070,0.744 ± 0.089,0.933 ± 0.015,5
1,Multiview + Ensemble,esm-33-gearnet-ensemble,0.661 ± 0.020,0.509 ± 0.036,0.904 ± 0.014,0.949 ± 0.012,5
2,Multiview + Pretrained + Ensemble,esm-33-gearnet-pretrained-ensemble,0.679 ± 0.024,0.531 ± 0.031,0.912 ± 0.014,0.947 ± 0.008,5
3,Multiview + + Boost All,esm-33-gearnet-aboost50,0.681 ± 0.043,0.531 ± 0.058,0.916 ± 0.015,0.940 ± 0.021,5
4,ESM + Pretrained + Ensemble,esm-t33-pretrained-ensemble,0.685 ± 0.041,0.549 ± 0.066,0.903 ± 0.062,0.935 ± 0.028,5
5,Multiview + Pretrained,esm-33-gearnet-pretrained,0.686 ± 0.044,0.569 ± 0.065,0.873 ± 0.027,0.929 ± 0.036,5
6,Multiview + Pretrained + Boost All,esm-33-gearnet-pretrained-freezelm-aboost50,0.692 ± 0.012,0.543 ± 0.020,0.923 ± 0.008,0.944 ± 0.017,5
7,ESM + Pretrained,esm-t33-pretrained,0.704 ± 0.023,0.606 ± 0.026,0.859 ± 0.033,0.935 ± 0.028,5
8,ESM,esm-t33,0.714 ± 0.021,0.597 ± 0.031,0.894 ± 0.030,0.954 ± 0.017,5
9,Multiview + Resiboost,esm-33-gearnet-rboost50,0.716 ± 0.018,0.583 ± 0.028,0.919 ± 0.011,0.946 ± 0.009,5


In [8]:
get_stat_df('bosutinib.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('bosutinib.csv', groups=groups)

| model_key                                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:--------------------------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet-pretrained-freezeall         | 0.683 ± 0.032 | 0.647 ± 0.077 | 0.777 ± 0.113 | 0.948 ± 0.012 |
| esm-t33-pretrained-freezelm                 | 0.687 ± 0.019 | 0.640 ± 0.085 | 0.794 ± 0.130 | 0.963 ± 0.005 |
| esm-33-gearnet-pretrained-freezelm-aboost05 | 0.711 ± 0.051 | 0.590 ± 0.079 | 0.898 ± 0.013 | 0.953 ± 0.010 |
| esm-t33-pretrained-ensemble                 | 0.718 ± 0.029 | 0.620 ± 0.034 | 0.871 ± 0.040 | 0.964 ± 0.006 |
| esm-t33-rboost50                            | 0.731 ± 0.018 | 0.620 ± 0.027 | 0.899 ± 0.018 | 0.952 ± 0.015 |
| esm-t33-pretrained-rboost50                 | 0.735 ± 0.019 | 0.620 ± 0.014 | 0.908 ± 0.036 | 0.972 ± 0.007 |
| esm-33-gearnet-pretrained-freezelm-aboost10 | 0.739 ± 0.027 | 0.620 ± 0.045 | 0.917 ± 0.016 | 0.954 ± 

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,ESM + Pretrained + Ensemble,esm-t33-pretrained-ensemble,0.718 ± 0.029,0.620 ± 0.034,0.871 ± 0.040,0.964 ± 0.006,5
1,ESM + Resiboost,esm-t33-rboost10,0.746 ± 0.005,0.673 ± 0.035,0.862 ± 0.036,0.956 ± 0.025,5
2,ESM + Pretrained,esm-t33-pretrained,0.746 ± 0.031,0.667 ± 0.026,0.870 ± 0.036,0.964 ± 0.006,5
3,ESM + Ensemble,esm-t33-ensemble,0.747 ± 0.013,0.643 ± 0.028,0.902 ± 0.015,0.946 ± 0.023,5
4,Multiview,esm-33-gearnet,0.749 ± 0.041,0.677 ± 0.066,0.866 ± 0.043,0.958 ± 0.021,5
5,Multiview + Pretrained + Ensemble,esm-33-gearnet-pretrained-freezelm-ensemble,0.753 ± 0.023,0.667 ± 0.041,0.886 ± 0.022,0.963 ± 0.010,5
6,ESM,esm-t33,0.754 ± 0.019,0.683 ± 0.037,0.866 ± 0.027,0.928 ± 0.034,5
7,Multiview + Pretrained + Boost All,esm-33-gearnet-pretrained-freezelm-aboost50,0.755 ± 0.009,0.650 ± 0.012,0.911 ± 0.010,0.958 ± 0.013,5
8,Multiview + Ensemble,esm-33-gearnet-ensemble,0.756 ± 0.018,0.653 ± 0.027,0.907 ± 0.004,0.964 ± 0.013,5
9,Multiview + + Boost All,esm-33-gearnet-aboost50,0.760 ± 0.018,0.663 ± 0.022,0.904 ± 0.012,0.961 ± 0.015,5


In [9]:
get_stat_df('methotrexate.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('methotrexate.csv', groups=groups)

| model_key                                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:--------------------------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet-pretrained-freezeall         | 0.218 ± 0.041 | 0.242 ± 0.073 | 0.269 ± 0.084 | 0.705 ± 0.056 |
| esm-33-gearnet                              | 0.441 ± 0.040 | 0.329 ± 0.059 | 0.650 ± 0.120 | 0.716 ± 0.062 |
| esm-33-gearnet-ensemble                     | 0.456 ± 0.051 | 0.300 ± 0.064 | 0.741 ± 0.008 | 0.724 ± 0.068 |
| esm-33-gearnet-pretrained                   | 0.456 ± 0.053 | 0.321 ± 0.059 | 0.696 ± 0.072 | 0.745 ± 0.079 |
| esm-33-gearnet-pretrained-ensemble          | 0.466 ± 0.058 | 0.300 ± 0.061 | 0.768 ± 0.031 | 0.770 ± 0.070 |
| esm-t33-pretrained-freezelm                 | 0.469 ± 0.050 | 0.468 ± 0.143 | 0.538 ± 0.090 | 0.819 ± 0.063 |
| esm-33-gearnet-pretrained-freezelm-aboost05 | 0.474 ± 0.056 | 0.326 ± 0.048 | 0.732 ± 0.069 | 0.792 ± 

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,Multiview,esm-33-gearnet,0.441 ± 0.040,0.329 ± 0.059,0.650 ± 0.120,0.716 ± 0.062,5
1,Multiview + Ensemble,esm-33-gearnet-ensemble,0.456 ± 0.051,0.300 ± 0.064,0.741 ± 0.008,0.724 ± 0.068,5
2,Multiview + Pretrained + Ensemble,esm-33-gearnet-pretrained-freezelm-ensemble,0.483 ± 0.062,0.345 ± 0.065,0.718 ± 0.059,0.802 ± 0.080,5
3,Multiview + Pretrained,esm-33-gearnet-pretrained-freezelm,0.495 ± 0.079,0.395 ± 0.120,0.674 ± 0.049,0.797 ± 0.086,5
4,ESM + Pretrained,esm-t33-pretrained,0.514 ± 0.091,0.453 ± 0.142,0.643 ± 0.116,0.871 ± 0.049,5
5,ESM + Pretrained + Ensemble,esm-t33-pretrained-ensemble,0.521 ± 0.088,0.420 ± 0.133,0.711 ± 0.114,0.871 ± 0.049,5
6,Multiview + Pretrained + Boost All,esm-33-gearnet-pretrained-freezelm-aboost25,0.522 ± 0.058,0.373 ± 0.068,0.772 ± 0.020,0.807 ± 0.061,5
7,Multiview + Resiboost,esm-33-gearnet-rboost50,0.525 ± 0.059,0.390 ± 0.078,0.750 ± 0.015,0.784 ± 0.076,5
8,ESM,esm-t33,0.525 ± 0.079,0.482 ± 0.131,0.630 ± 0.102,0.881 ± 0.049,5
9,ESM + Resiboost,esm-t33-rboost10,0.558 ± 0.077,0.483 ± 0.131,0.697 ± 0.063,0.866 ± 0.040,5


In [10]:
get_stat_df('atpbind3d.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('atpbind3d.csv', groups=groups)

| model_key               | mcc           | sensitivity   | precision     | micro_auroc   |
|:------------------------|:--------------|:--------------|:--------------|:--------------|
| gearnet                 | 0.483 ± 0.011 | 0.405 ± 0.040 | 0.630 ± 0.039 | 0.872 ± 0.008 |
| bert                    | 0.499 ± 0.030 | 0.424 ± 0.026 | 0.640 ± 0.066 | 0.875 ± 0.020 |
| esm-33-gearnet-aboost05 | 0.657 ± 0.012 | 0.550 ± 0.020 | 0.820 ± 0.022 | 0.925 ± 0.009 |
| esm-t33-aboost50        | 0.658 ± 0.006 | 0.547 ± 0.007 | 0.825 ± 0.015 | 0.950 ± 0.005 |
| esm-t33-rboost50        | 0.664 ± 0.006 | 0.552 ± 0.019 | 0.831 ± 0.020 | 0.949 ± 0.004 |
| esm-t33-aboost10        | 0.666 ± 0.017 | 0.574 ± 0.025 | 0.806 ± 0.012 | 0.943 ± 0.003 |
| esm-t33                 | 0.667 ± 0.008 | 0.636 ± 0.014 | 0.733 ± 0.014 | 0.935 ± 0.007 |
| esm-33-gearnet-aboost10 | 0.667 ± 0.010 | 0.565 ± 0.014 | 0.821 ± 0.017 | 0.908 ± 0.008 |
| esm-t33-ensemble        | 0.668 ± 0.009 | 0.580 ± 0.014 | 0.803 ± 0.015 | 0.95

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,GearNet,gearnet,0.483 ± 0.011,0.405 ± 0.040,0.630 ± 0.039,0.872 ± 0.008,5
1,ESM,esm-t33,0.667 ± 0.008,0.636 ± 0.014,0.733 ± 0.014,0.935 ± 0.007,10
2,ESM + Ensemble,esm-t33-ensemble,0.668 ± 0.009,0.580 ± 0.014,0.803 ± 0.015,0.952 ± 0.003,5
3,Multiview,esm-33-gearnet,0.669 ± 0.010,0.623 ± 0.025,0.753 ± 0.022,0.928 ± 0.006,10
4,Multiview + Ensemble,esm-33-gearnet-ensemble,0.669 ± 0.012,0.571 ± 0.016,0.816 ± 0.017,0.939 ± 0.004,5
5,Multiview + + Boost All,esm-33-gearnet-aboost25,0.676 ± 0.012,0.583 ± 0.018,0.816 ± 0.016,0.911 ± 0.008,10
6,ESM + + Boost All,esm-t33-aboost25,0.679 ± 0.013,0.592 ± 0.009,0.812 ± 0.020,0.951 ± 0.004,5
7,ESM + Resiboost,esm-t33-rboost10,0.683 ± 0.008,0.611 ± 0.011,0.796 ± 0.014,0.945 ± 0.003,5
8,Multiview + Resiboost,esm-33-gearnet-rboost05,0.689 ± 0.007,0.688 ± 0.010,0.723 ± 0.009,0.924 ± 0.006,5
